## Load colmap DB

In [1]:
from typing import Union, List, Dict, Tuple
import os
from collections import defaultdict
os.chdir("..")
from copy import deepcopy
import torch
import cv2
import numpy as np
import matplotlib.cm as cm
import sqlite3
import numpy as np

from colmap_database import (
    COLMAPDatabase, array_to_blob, blob_to_array, image_ids_to_pair_id, pair_id_to_image_ids
)
from src.utils.plotting import make_matching_figure
from src.loftr import LoFTR, default_cfg
from paralleldomain import AnyPath

In [2]:
db_path = "/home/michael/datasets/Tokyo_Ginza_alley_ham_store_12_16/distorted/database.db"
db = COLMAPDatabase.connect(db_path)

In [3]:
image_map = db.execute("SELECT image_id, name FROM images").fetchall()
# image_ids = [im[0] for im in image_map]
# image_names = [im[1] for im in image_map]

In [4]:
image_path = AnyPath("/home/michael/datasets/Tokyo_Ginza_alley_ham_store_12_16/originalImages")

In [5]:
def parse_image_names(image_map):
    for idx,(img0_id, img0_name) in enumerate(image_map):
        for img1_id, img1_name in image_map[idx+1:]:
            pair_id = image_ids_to_pair_id(img0_id,img1_id)
            yield pair_id, img0_id, img0_name, img1_id, img1_name

def preprocess_images(img_path: Union[AnyPath,str], img0_name: Union[AnyPath,str], img1_name: Union[AnyPath,str], dims=(768,512)) -> Dict[str,torch.Tensor]:
    img0_pth = AnyPath(img_path) / img0_name
    img1_pth = AnyPath(img_path) / img1_name
    img0_raw = cv2.resize(cv2.imread(str(img0_pth), cv2.IMREAD_GRAYSCALE),dims)
    img1_raw = cv2.resize(cv2.imread(str(img1_pth), cv2.IMREAD_GRAYSCALE),dims)
    # Check that input size is divisible by 8
    img0_raw = cv2.resize(img0_raw, (img0_raw.shape[1]//8*8, img0_raw.shape[0]//8*8))
    img1_raw = cv2.resize(img1_raw, (img1_raw.shape[1]//8*8, img1_raw.shape[0]//8*8))

    img0 = torch.from_numpy(img0_raw)[None][None].cuda() / 255.
    img1 = torch.from_numpy(img1_raw)[None][None].cuda() / 255.
    batch = {'image0': img0, 'image1': img1}

    return batch

In [6]:
# Instantiate the model
# The default config uses dual-softmax.
# The outdoor and indoor models share the same config.
# You can change the default values like thr and coarse_match_type.
matcher = LoFTR(config=default_cfg)
matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
matcher = matcher.eval().cuda()

In [7]:
keypoint_blob_dict = defaultdict(dict)

In [14]:
#TODO: rescale to full images?
count = 0
for pair_id, img0_id, img0_name, img1_id, img1_name in parse_image_names(image_map):

    batch = preprocess_images(img_path=image_path, img0_name=img0_name, img1_name=img1_name)
    # Inference with LoFTR and get prediction
    with torch.no_grad():
        matcher(batch)
        mkpts0 = batch['mkpts0_f'].cpu().numpy()
        mkpts1 = batch['mkpts1_f'].cpu().numpy()
        # mconf = batch['mconf'].cpu().numpy()
    
    match_table_entry = []
    idx0 = 0
    idx1 = 0
    for coords0,coords1 in zip(mkpts0.astype(int),mkpts1.astype(int)):
        # Each keypoint is added to keypoint_blob_dict[img_id].
        # The point's index in that list is entered into the matches table.
        feat_idx0 = keypoint_blob_dict[img0_id].setdefault(tuple(coords0),idx0)
        feat_idx1 = keypoint_blob_dict[img1_id].setdefault(tuple(coords1),idx1)
        if idx0 == feat_idx0:
            idx0 += 1
        if idx1 == feat_idx1:
            idx1 += 1
        # Add to matching table blob
        match_table_entry.append([feat_idx0,feat_idx1])
    # Write matching blob to matches table
    db.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + mkpts0.shape + (array_to_blob(np.array(match_table_entry)),
            ),)
    count += 1
    if count == 15:
        break

In [15]:
# Write the keypoint table
for image_id,keypoint_dict in keypoint_blob_dict.items():
    keypoints = np.array(list(keypoint_dict.keys()))
    db.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),),
        )

In [27]:
keypoints

array([[510,  97],
       [496, 297],
       [353, 231],
       [736,  24],
       [294,  40],
       [ 33, 224],
       [ 48, 231],
       [ 17, 225],
       [ 32, 231],
       [ 40, 232],
       [ 47, 239],
       [303, 238],
       [321, 229],
       [313, 223],
       [474, 185],
       [192,  71],
       [ 16, 233],
       [ 39, 242],
       [ 46, 245],
       [673, 344],
       [344, 280],
       [311, 240],
       [472, 208],
       [128,  64],
       [ 15, 240],
       [ 55, 256],
       [688, 351],
       [350, 286],
       [392, 240],
       [110, 423],
       [ 34, 273],
       [335, 262],
       [ 33, 281],
       [ 48, 286],
       [ 96, 447],
       [690, 336],
       [ 17, 285],
       [ 24, 286],
       [ 32, 288],
       [ 32, 438],
       [ 39, 439],
       [ 65, 288],
       [ 32, 296],
       [ 55, 304],
       [ 22, 303],
       [ 31, 304],
       [ 39, 305],
       [ 55, 311],
       [439, 376],
       [455, 375],
       [736, 407],
       [519, 215],
       [ 14,

In [46]:
#TODO: round matches to nearest pixel? 
# keypoint entry for img 1
np.array(list(keypoint_dict.keys())).dtype

dtype('int64')

In [22]:
pair_id_to_image_ids(pair_id)

(109.0, 1187)

In [48]:
match_arrr = blob_to_array(match_blob,dtype=np.int64,shape=(match_rows,match_cols))
keypt_arr = blob_to_array(keypt_blob,dtype=np.int64,shape=(keypt_rows,keypt_cols))

In [56]:
rows,cols,match_blob = db.execute("SELECT rows,cols,data from matches WHERE pair_id=?",(pair_id,)).fetchone()
blob_to_array(match_blob,dtype=np.int64,shape=(match_rows,match_cols))

array([[   0,    0],
       [   0,    1],
       [   1,    2],
       ...,
       [1620, 3530],
       [1621, 3531],
       [1622, 3532]])

In [51]:
match_arrr

array([[   0,    0],
       [   0,    1],
       [   1,    2],
       ...,
       [1620, 3530],
       [1621, 3531],
       [1622, 3532]])

In [24]:
pair_id, match_rows, match_cols, match_blob = db.execute("SELECT * FROM matches;").fetchone()

In [25]:
img_id, keypt_rows, keypt_cols, keypt_blob = db.execute("SELECT * FROM keypoints;").fetchone()

### Matching

In [ ]:
db.execute("PRAGMA table_info('matches')").fetchall()

In [ ]:
rows, cols = mkpts0.shape

## Indoor Example

In [ ]:
from src.loftr import LoFTR, default_cfg

# The default config uses dual-softmax.
# The outdoor and indoor models share the same config.
# You can change the default values like thr and coarse_match_type.
_default_cfg = deepcopy(default_cfg)
_default_cfg['coarse']['temp_bug_fix'] = True  # set to False when using the old ckpt
matcher = LoFTR(config=_default_cfg)
matcher.load_state_dict(torch.load("weights/indoor_ds_new.ckpt")['state_dict'])
matcher = matcher.eval().cuda()

In [ ]:

# Load example images
img0_pth = img_path+
img1_pth = "assets/scannet_sample_images/scene0711_00_frame-001995.jpg"

img0_raw = cv2.imread(img0_pth, cv2.IMREAD_GRAYSCALE)
img1_raw = cv2.imread(img1_pth, cv2.IMREAD_GRAYSCALE)
img0_raw = cv2.resize(img0_raw, (640, 480))
img1_raw = cv2.resize(img1_raw, (640, 480))

img0 = torch.from_numpy(img0_raw)[None][None].cuda() / 255.
img1 = torch.from_numpy(img1_raw)[None][None].cuda() / 255.
batch = {'image0': img0, 'image1': img1}

# Inference with LoFTR and get prediction
with torch.no_grad():
    matcher(batch)
    mkpts0 = batch['mkpts0_f'].cpu().numpy()
    mkpts1 = batch['mkpts1_f'].cpu().numpy()
    mconf = batch['mconf'].cpu().numpy()

In [ ]:
# Draw
color = cm.jet(mconf)
text = [
    'LoFTR',
    'Matches: {}'.format(len(mkpts0)),
]
fig = make_matching_figure(img0_raw, img1_raw, mkpts0, mkpts1, color, text=text)

## Outdoor Example

In [ ]:
# Draw
color = cm.jet(mconf)
text = [
    'LoFTR',
    'Matches: {}'.format(len(mkpts0)),
]
fig = make_matching_figure(img0_raw, img1_raw, mkpts0, mkpts1, color, text=text)

In [ ]:
mkpts1.max(axis=0)

In [ ]:
img0_raw.shape